In [109]:
# !pip install scikit-learn

In [110]:
import os
import shutil

In [ ]:
DATASET_PATH: str = 'E:\Downloads\qweqwewqewq-02-01.v1-qweqwewqewq-02-01.yolov11'

In [112]:
def create_dir(dir_name: str) -> None:
    new_dir: str = os.path.join(DATASET_PATH, dir_name)
    
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
        os.mkdir(os.path.join(new_dir, 'images'))
        os.mkdir(os.path.join(new_dir, 'labels'))
        
        print(f'Folder {dir_name} created!') 
        return
    
    print(f'Folder {dir_name} already exists!') 
    

In [113]:
create_dir('test')
create_dir('valid')

Folder test already exists!
Folder valid already exists!


In [114]:
def get_max_class(values: list[str]) -> str:
    classes: dict[str, int] = {}
    for value in values:
        value = value.split()[0]
        if classes.get(value) is None:
            classes[value] = 0
        classes[value] += 1
    if not classes: return 'Null'
    return max(classes.items(), key=lambda x: x[1])[0]

In [115]:
TARGET_DIR: str = os.path.join(DATASET_PATH, 'train/labels')
CLASSES_PATH: dict[str, str] = {}

for label in os.listdir(TARGET_DIR):
    label_path: str = os.path.join(TARGET_DIR, label)
    with open(label_path, 'r', encoding='utf-8') as file:
        label_key = get_max_class(file.readlines())
        
        if CLASSES_PATH.get(label_key) is None:
            CLASSES_PATH[label_key] = []
        
        CLASSES_PATH[label_key].append(label_path)
        

In [ ]:
def find_image_from_dataset(images_path: str, target: str) -> str:
    for file in os.listdir(images_path):
        for i in ['jpg', 'png', 'jpeg']:
            if file == target+f'.{i}':
                return file
    return None


def get_root_dataset_path(path: str) -> str:
    path = os.path.normpath(path).split(os.sep)
    index = path.index('labels')
    return '\\'.join(path[:index])


def move_files(target_path, files_path: list) -> None:
    for file in files_path:
        shutil.move(file, os.path.join(DATASET_PATH, target_path, 'labels'))
        image_path: str = os.path.join(get_root_dataset_path(file), 'images')
        image = find_image_from_dataset(image_path, os.path.splitext(os.path.basename(file))[0])
        if image is None:
            print('Image not found, continue!}')
            continue
        shutil.move(
            os.path.join(image_path, image), 
            os.path.join(DATASET_PATH, target_path, 'images')
        )
        
        

In [117]:
from sklearn.model_selection import train_test_split

for key in CLASSES_PATH.keys():
    train, temp = train_test_split(CLASSES_PATH[key], test_size=0.3,  random_state=42, shuffle=True)
    valid, test = train_test_split(temp, test_size=0.5,  random_state=42, shuffle=True)
    
    move_files('test', test)
    move_files('valid', valid)
    
    